In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd


In [11]:
# Load ANN model, scaler and encoder pickle files
model=load_model('churn_model.h5')

with open('onehotencoder_geo.pkl','rb') as f:
    geo_encoder=pickle.load(f)

with open('label_encoder_gender.pkl','rb') as f:
    gen_encoder=pickle.load(f)

with open('scaler.pkl','rb') as f:
    scaler=pickle.load(f)



In [5]:
input_data = {
    'CreditScore': 700,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 42,
    'Tenure': 2,
    'Balance': 65000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 51000
}


In [15]:
# Do the encoding for Geography
geo_encoded = geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=geo_encoder.get_feature_names_out(['Geography']))
geo_encoded_df


d:\Proj1\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [16]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,France,Female,42,2,65000,2,1,1,51000


In [17]:
#Encode Gender
input_df['Gender']=gen_encoder.transform(input_df['Gender'])

In [18]:
# Combining Data
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,0,42,2,65000,2,1,1,51000,1.0,0.0,0.0


In [19]:
# Scaling
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 0.50178794, -1.09499335,  0.29493847, -1.04241787, -0.17775624,
         0.80843615,  0.64920267,  0.97481699, -0.85944554,  1.00150113,
        -0.57946723, -0.57638802]])

In [20]:
# Prediction
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


array([[0.09971146]], dtype=float32)

In [21]:
prediction_prob=prediction[0][0]
prediction_prob

if prediction_prob>0.5:
    print("Customer is likely to churn")
else:
    print("Customer is likely to stay")

Customer is likely to stay
